<a href="https://colab.research.google.com/github/siting1206/NLP_HW2/blob/main/BM25(mini_dp).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from gensim import corpora
from gensim.summarization import bm25
from nltk.stem.porter import PorterStemmer  
from collections import namedtuple
import regex as re
import heapq
p_stemmer = PorterStemmer()

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
Tag = namedtuple("Tag", ["snippets", "question", "answer"])

# Load train/validation dataset definition
def load_sequences(filename, sep="\t", notypes=False, test_data=False):
    sequences = []
    with open(filename) as fp:
        seq = []
        for line in fp:
            line = line.strip()
            if line:
                line = line.replace(" ||| ", "\t")
                line = line.split(sep)
                seq.append(Tag(*line))
            else:
                sequences.append(seq)
                seq = []
        if seq:
            sequences.append(seq)
    return sequences

In [13]:
train_sequences = load_sequences("drive/MyDrive/NLP_assignment2/Dataset/mini.txt", sep="\t", notypes=True)

In [29]:
train_snippets = [[t[0] for t in seq] for seq in (train_sequences)] # length = 99820
train_question = [[t[1] for t in seq] for seq in (train_sequences)]
train_answer = [[t[2] for t in seq] for seq in (train_sequences)]

In [15]:
def sort_index(lst, rev=True):
    index = range(len(lst))
    s = sorted(index, reverse=rev, key=lambda i: lst[i])
    return s

In [16]:
def snippets_filter(snippets, question, trn_s = []):
  
  for i in range(0, len(snippets)):
    snippets[i] = re.findall('<s> (.*?) </s>', snippets[i])
    question[i] = question[i].split()
    snippets_list =[]
    
    for a in snippets[i]:
      a_split = a.replace('?',' ').replace('(',' ').replace(')',' ').split(' ')
      # 詞干提取(e.g. assignment->assign, words->word)
      stemmed_tokens = [p_stemmer.stem(i) for i in a_split] 
      snippets_list.append(stemmed_tokens)
    query =[question[i]]
    query_stemmed = [p_stemmer.stem(i) for i in query[0]]
    # bm25模型
    bm25Model = bm25.BM25(snippets_list)
    # 計算各個snippets與answer相關性
    average_idf = sum(map(lambda k: float(bm25Model.idf[k]), bm25Model.idf.keys())) / len(bm25Model.idf.keys())
    scores = bm25Model.get_scores(query_stemmed,average_idf)
    max_index = sort_index(scores)[:3]
    # print(max_index)
    s = []
    for m in range(0, len(max_index)):
      n = max_index[m]
      s.append(snippets[i][n])
    trn_s.append(s)
  return trn_s, question

In [30]:
trn_s, trn_q = snippets_filter(train_snippets[0], train_question[0], trn_s = [])
print(len(trn_s), len(trn_q))

4 4


In [25]:
print(train_question)
print(train_answer)

[[['last', '8', 'years', 'life', ',', 'galileo', 'house', 'arrest', 'espousing', 'man', "'s", 'theory'], ['2', '1912', 'olympian', 'football', 'star', 'carlisle', 'indian', 'school', '6', 'mlb', 'seasons', 'reds', ',', 'giants', 'braves'], ['city', 'yuma', 'state', 'record', 'average', '4', ',', '055', 'hours', 'sunshine', 'year'], ['signer', 'dec', 'indep', ',', 'framer', 'constitution', 'mass', ',', 'second', 'president', 'united', 'states']]]
[['copernicus', 'jim thorpe', 'arizona', 'john adams']]


In [33]:
f = open("/content/drive/MyDrive/NLP_assignment2/Dataset/mini_ext.txt", "w")
f.write("snippet\tquesion\tanswer\n")
for i in range(0, len(trn_s)):
  for j in range(0, len(trn_s[i])):
    f.write(trn_s[i][j] + "\t")
    for k in range(0, len(trn_q[i])):
      f.write(trn_q[i][k] + " ")
    f.write("\t" + train_answer[0][i] + "\n")
f.close()